# Data Cleanup
This file prepares the fits files for the automatic reduction pipeline process. For more info, see the README in parent directory.

In [1]:
# Imports for the job

# for fits file operations
from astropy.io import fits
# for working with file systems
from pathlib import Path
# for tesitng the result of our work
from ccdproc import ImageFileCollection

Specify the working directories. Create target if not already there.

In [2]:
# Raw data directory
data_raw = Path('data')

# Cleaned data target directory
data_cleaned = Path('data-cleaned')
# Make sure this directory exists; exist_ok means this line does nothing if
# already created this folder.
data_cleaned.mkdir(exist_ok=True)

### Cleanup process.

Here, it will be important how each file is called. Need to **EDIT** per the specific collection of images.

In [3]:
# Include image type in header for fits - useful for filtering during
# processing. Save the modified files in a new directory called data-cleaned.
# Currently, it is done manually ONLY.

# list of (image_name_pattern, num_image, type_image) to be prepared
dict_files = [
    ('Dark-5s-00{}.FIT', 5, 'DARK'),
    ('Dark-180s-00{}.FIT', 3, 'DARK'),
    ('gCas-360s-00{}.FIT', 3, 'LIGHT'),
    ('Lamp-5s-00{}.FIT', 3, 'CALIB'),
    ('vega-180s-00{}.FIT', 4, 'LIGHT')
]

# generalised function definition - rewrites the files with header added
def assign_imagetyp(pattern, num_img, imagetyp):
    """Wrapper of writing 'imagetyp' to fits and saving in new directory.

    Parameters
    ----------
    pattern : string
        Pattern to match the names. Has to have a {} for formatting.
    num_img : int
        Number of images following the pattern. NOTE: currently, only 
        implemented for number of images LESS than 10!
    imagetyp : string
        The imagetyp to be assigned to the files following the naming pattern.
    """
    image_name_read = [pattern.format(x+1) for x in range(num_img)]
    image_name_write = [(pattern.format(x+1)).lower() for x in range(num_img)]

    for i in range(num_img):
        with fits.open(data_raw / image_name_read[i]) as hdul:
            hdr = hdul[0].header
            hdr['imagetyp'] = imagetyp
            hdr['bunit'] = 'adu' # needed for image arithmetic with ccdproc
            hdul.writeto(data_cleaned / image_name_write[i], overwrite=True)

# apply function to our images
for entry in dict_files:
    assign_imagetyp(entry[0], entry[1], entry[2])

In [4]:
# Check we made a good job.
# Again, modify as fit.

im_collection = ImageFileCollection(data_cleaned)
im_collection.summary

file,simple,bitpix,naxis,naxis1,naxis2,object,telescop,instrume,observer,date-obs,bscale,bzero,exptime,ccd-temp,xpixsz,ypixsz,xbinning,ybinning,xorgsubf,yorgsubf,egain,focallen,aptdia,aptarea,cblack,cwhite,pedestal,datamax,sbstdver,swacquir,swcreate,filter,snapshot,date,resmode,expstate,response,note,imagetyp,bunit
str17,bool,int32,int32,int32,int32,str1,str17,str11,str15,str23,int32,int32,float64,float64,float64,float64,int32,int32,int32,int32,float64,float64,float64,float64,int32,int32,int32,int32,str21,str26,str39,str4,int32,str10,int32,str3,float64,str31,str5,str3
dark-180s-001.fit,True,16,2,1530,1020,,Unknown Telescope,SBIG ST-402,Observer's Name,2022-07-25T22:01:30.000,1,32768,180.0,-7.301899046322,9.0,9.0,1,1,0,0,1.54,2032.0,203.2,27868.912004,996,1125,-100,65535,SBFITSEXT Version 1.0,WinOPS Ver 5.47 Build 6-NT,SBIG Win CCDOPS Version 5.47 Build 6-NT,None,1,2022-07-25,0,125,3000.0,Local time:7/26/2022 at 1:01:30,DARK,adu
dark-180s-002.fit,True,16,2,1530,1020,,Unknown Telescope,SBIG ST-402,Observer's Name,2022-07-25T22:04:33.000,1,32768,180.0,-7.301899046322,9.0,9.0,1,1,0,0,1.54,2032.0,203.2,27868.912004,996,1125,-100,65535,SBFITSEXT Version 1.0,WinOPS Ver 5.47 Build 6-NT,SBIG Win CCDOPS Version 5.47 Build 6-NT,None,1,2022-07-25,0,125,3000.0,Local time:7/26/2022 at 1:04:33,DARK,adu
dark-180s-003.fit,True,16,2,1530,1020,,Unknown Telescope,SBIG ST-402,Observer's Name,2022-07-25T22:07:35.000,1,32768,180.0,-7.301899046322,9.0,9.0,1,1,0,0,1.54,2032.0,203.2,27868.912004,996,1125,-100,65535,SBFITSEXT Version 1.0,WinOPS Ver 5.47 Build 6-NT,SBIG Win CCDOPS Version 5.47 Build 6-NT,None,1,2022-07-25,0,125,3000.0,Local time:7/26/2022 at 1:07:35,DARK,adu
dark-5s-001.fit,True,16,2,1530,1020,,Unknown Telescope,SBIG ST-402,Observer's Name,2022-07-26T01:21:38.000,1,32768,5.0,-7.301899046322,9.0,9.0,1,1,0,0,1.54,2032.0,203.2,27868.912004,1000,1129,-100,65535,SBFITSEXT Version 1.0,WinOPS Ver 5.47 Build 6-NT,SBIG Win CCDOPS Version 5.47 Build 6-NT,None,1,2022-07-26,0,125,3000.0,Local time:7/26/2022 at 4:21:38,DARK,adu
dark-5s-002.fit,True,16,2,1530,1020,,Unknown Telescope,SBIG ST-402,Observer's Name,2022-07-26T01:21:46.000,1,32768,5.0,-7.301899046322,9.0,9.0,1,1,0,0,1.54,2032.0,203.2,27868.912004,1000,1129,-100,65535,SBFITSEXT Version 1.0,WinOPS Ver 5.47 Build 6-NT,SBIG Win CCDOPS Version 5.47 Build 6-NT,None,1,2022-07-26,0,125,3000.0,Local time:7/26/2022 at 4:21:46,DARK,adu
dark-5s-003.fit,True,16,2,1530,1020,,Unknown Telescope,SBIG ST-402,Observer's Name,2022-07-26T01:21:54.000,1,32768,5.0,-7.301899046322,9.0,9.0,1,1,0,0,1.54,2032.0,203.2,27868.912004,1000,1129,-100,65535,SBFITSEXT Version 1.0,WinOPS Ver 5.47 Build 6-NT,SBIG Win CCDOPS Version 5.47 Build 6-NT,None,1,2022-07-26,0,125,3000.0,Local time:7/26/2022 at 4:21:54,DARK,adu
dark-5s-004.fit,True,16,2,1530,1020,,Unknown Telescope,SBIG ST-402,Observer's Name,2022-07-26T01:22:01.000,1,32768,5.0,-7.301899046322,9.0,9.0,1,1,0,0,1.54,2032.0,203.2,27868.912004,1000,1129,-100,65535,SBFITSEXT Version 1.0,WinOPS Ver 5.47 Build 6-NT,SBIG Win CCDOPS Version 5.47 Build 6-NT,None,1,2022-07-26,0,125,3000.0,Local time:7/26/2022 at 4:22:01,DARK,adu
dark-5s-005.fit,True,16,2,1530,1020,,Unknown Telescope,SBIG ST-402,Observer's Name,2022-07-26T01:22:09.000,1,32768,5.0,-7.301899046322,9.0,9.0,1,1,0,0,1.54,2032.0,203.2,27868.912004,1000,1129,-100,65535,SBFITSEXT Version 1.0,WinOPS Ver 5.47 Build 6-NT,SBIG Win CCDOPS Version 5.47 Build 6-NT,None,1,2022-07-26,0,125,3000.0,Local time:7/26/2022 at 4:22:09,DARK,adu
gcas-360s-001.fit,True,16,2,1530,1020,,Unknown Telescope,SBIG ST-402,Observer's Name,2022-07-26T00:19:56.000,1,32768,360.0,-6.888051704092,9.0,9.0,1,1,0,0,1.54,2032.0,203.2,27868.912004,870,2339,-100,65535,SBFITSEXT Version 1.0,WinOPS Ver 5.47 Build 6-NT,SBIG Win CCDOPS Version 5.47 Build 6-NT,None,1,2022-07-26,0,125,3000.0,Local time:7/26/2022 at 3:19:56,LIGHT,adu
